In \[1\]:

    unsw_rdd = sc.textFile("/home/wednesday_group/Downloads/UNSW-NB15.csv")

In \[2\]:

    unsw_rdd.take(2)

Out\[2\]:

    ['59.166.0.3,56716,149.171.126.8,143,tcp,FIN,0.82546002,7812,16236,31,29,30,32,-,75090.25,156111.73,122,126,255,255,2751097753,2748686736,64,129,0,0,445.25928,474.9451,1421970774,1421970775,6.8190908,6.599896,5.9700001E-4,4.6899999E-4,0.000128,0,0,0,0,0,2,7,1,4,1,1,1,,0',
     '59.166.0.0,43467,149.171.126.6,49729,tcp,FIN,0.101815,4238,65628,31,29,7,30,-,328438.84,5087030.5,72,74,255,255,961515433,3225510659,59,887,0,0,0,91.579567,1421970775,1421970775,1.429493,1.387192,0.00068,5.4600002E-4,0.000134,0,0,0,0,0,7,4,1,6,1,1,1,,0']

In \[57\]:

    from pyspark.mllib.regression import LabeledPoint
    from numpy import array

    def parse_interaction(line):
        line_split = line.split(",")
        # leave_out = [1,2,3,41]
        clean_line_split = [6,7,8,9,10,11,12,16,17,18,19,20,21,22,23,24,25,26,27,30,31,32,33,34,36,43,44,45,46,40,41,42,48,35]
        attack = 1.0
        if line_split[47]== '':
            attack = 0.0
        return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

    main_data = unsw_rdd.map(parse_interaction)

In \[58\]:

    main_data.take(2)

Out\[58\]:

    [LabeledPoint(0.0, [6.0,7.0,8.0,9.0,10.0,11.0,12.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,43.0,44.0,45.0,46.0,40.0,41.0,42.0,48.0,35.0]),
     LabeledPoint(0.0, [6.0,7.0,8.0,9.0,10.0,11.0,12.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,43.0,44.0,45.0,46.0,40.0,41.0,42.0,48.0,35.0])]

In \[59\]:

    [train_data, test_data] = main_data.randomSplit([0.7, 0.3], seed = 1234)

In \[60\]:

    #[train_data.count(), test_data.count()]

In \[61\]:

    from pyspark.mllib.classification import LogisticRegressionWithLBFGS
    from time import time

    # Build the model
    t0 = time()
    logit_model = LogisticRegressionWithLBFGS.train(train_data)
    tt = time() - t0 # to know how long it takes to complete Regression analysis

    print ("Classifier trained in {} seconds".format(round(tt,3)))

    Classifier trained in 124.097 seconds

In \[62\]:

    labels_and_preds_train = train_data.map(lambda a: (abs(a.label)-abs(logit_model.predict(a.features))))

In \[63\]:

    labels_and_preds_train.take(10)

Out\[63\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[64\]:

    # Model accuracy on training data
    t0 = time()
    train_accuracy = labels_and_preds_train.filter(lambda a: a==0).count() / float(train_data.count())
    tt = time() - t0
    print ("Prediction made in {} seconds. Training accuracy is {}".format(round(tt,3), round(train_accuracy,4)))

    Prediction made in 114.839 seconds. Training accuracy is 0.8736

In \[65\]:

    labels_and_preds_test = test_data.map(lambda a: (abs(a.label)-abs(logit_model.predict(a.features))))

In \[66\]:

    labels_and_preds_test.take(10)

Out\[66\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[67\]:

    # model accuracy for test data
    t0 = time()
    test_accuracy = labels_and_preds_test.filter(lambda a: a==0).count() / float(test_data.count())
    tt = time() - t0
    print("Prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

    Prediction made in 243.242 seconds. Testing accuracy is 0.8734

In \[ \]: